*For Part-B, we could not save the best model as done in part-A due to some WandB errors. Hence, we manually find the best model from the plots and re-train it and evaluate it on the test-set.* **Note**: We have **not** used the test-set till now.

In [2]:
# import statements
from math import ceil
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow_addons.optimizers import RectifiedAdam, Lookahead
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.xception import Xception

In [3]:
# some fixed parameters
# this code assumes that the dataset is in the same directory as the script
# we fix the epochs as 5, as it takes really long for training
BASE_PATH = "./inaturalist_12K"
IMG_SIZE = (224, 224)
EPOCHS = 5

### BEST HYPERPARAMETERS ###
BASE_MODEL = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(*IMG_SIZE, 3))
BATCH_SIZE = 64
FREEZE = 0.66
LEARNING_RATE = 0.0001
WEIGHT_DECAY = 0.0001
DATA_AUGMENTATION = False

In [4]:
# a function for reading data from the given directory structure
# we put aside 10% data for validation and  rescale our image pixel values by 1/255
# returns the train, validation and test generators which can be passed to the model.fit()/model.evaluate() method
def get_data_generators(data_augmentation=True, batch_size=32, img_size=IMG_SIZE):
    if data_augmentation:
        # the following augmentation techniques are used
        data = ImageDataGenerator(rescale=1/255,
                                  samplewise_center=True,
                                  samplewise_std_normalization=True,
                                  validation_split=0.1,
                                  shear_range=0.25,
                                  zoom_range=[0.25, 1.25],
                                  width_shift_range=0.25,
                                  height_shift_range=0.25,
                                  horizontal_flip=True,
                                  rotation_range=60)
    else:
        data = ImageDataGenerator(rescale=1/255,
                                  validation_split=0.1)
        
    # this time read the test-data as well
    # but for evaluation we do not do any augmentation on the test data
    test_data = ImageDataGenerator(rescale=1/255)

    # here the class_mode is specified as sparse
    # this means the targets are specified as whole numbers (ex. 0, 1, 2 etc) instead of one-hot vectors
    # it is bit memory efficient this way
    train_gen = data.flow_from_directory(f"{BASE_PATH}/train",
                                         target_size=img_size,
                                         batch_size=batch_size,
                                         color_mode="rgb",
                                         class_mode="sparse",
                                         shuffle=True,
                                         seed=123,
                                         subset="training")

    validation_gen = data.flow_from_directory(f"{BASE_PATH}/train",
                                              target_size=img_size,
                                              batch_size=batch_size,
                                              color_mode="rgb",
                                              class_mode="sparse",
                                              shuffle=True,
                                              seed=123,
                                              subset="validation")
    
    test_gen = test_data.flow_from_directory(f"{BASE_PATH}/val",
                                             target_size=img_size,
                                             batch_size=batch_size,
                                             color_mode="rgb",
                                             class_mode="sparse",
                                             shuffle=True,
                                             seed=123)
                                              
    return train_gen, validation_gen, test_gen

In [5]:
# get the train, validation and test data
train_gen, validation_gen, test_gen = get_data_generators(DATA_AUGMENTATION, BATCH_SIZE)

Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


In [6]:
# freeze the specified fraction of layers
# by setting their trainable variable to False
# by default, all layers have trainable set as True
N = len(BASE_MODEL.layers)
for layer in BASE_MODEL.layers[: ceil(FREEZE*N)]:
    layer.trainable = False

# build the model
model = Sequential()
model.add(BASE_MODEL)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation="swish"))
model.add(Dropout(0.25))
model.add(Dense(10, activation="softmax"))

# we use a new optimizer Lookahead + Radam (a.k.a RANGER) with specified learning_rate and weight_decay
# this optimizer has proved to converge really fast and generalize well
# given the limited number of epochs we were able to make, we tried to use the most efficient optimizer to get the best results
OPTIM = Lookahead(RectifiedAdam(learning_rate=LEARNING_RATE, weight_decay=WEIGHT_DECAY, amsgrad=True))
model.compile(optimizer=OPTIM, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
# define the early stopping
# we stop the training if the val_accuracy drops (by 1e-4 or higher) continuously for 4 times
early_stop = EarlyStopping(monitor='val_accuracy', min_delta=1e-4, patience=4, restore_best_weights=True)
# fit the data to the model
model.fit(train_gen, epochs=EPOCHS, validation_data=validation_gen, callbacks=[early_stop])

Epoch 1/5
141/141 [==============================] - 161s 794ms/step - loss: 1.9167 - accuracy: 0.3954 - val_loss: 1.0743 - val_accuracy: 0.7247
Epoch 2/5
141/141 [==============================] - 106s 745ms/step - loss: 0.8892 - accuracy: 0.7517 - val_loss: 0.7265 - val_accuracy: 0.7828
Epoch 3/5
141/141 [==============================] - 105s 744ms/step - loss: 0.5652 - accuracy: 0.8217 - val_loss: 0.6478 - val_accuracy: 0.7848
Epoch 4/5
141/141 [==============================] - 105s 744ms/step - loss: 0.3765 - accuracy: 0.8820 - val_loss: 0.6535 - val_accuracy: 0.7938
Epoch 5/5
141/141 [==============================] - 105s 739ms/step - loss: 0.2134 - accuracy: 0.9346 - val_loss: 0.6902 - val_accuracy: 0.8028


In [7]:
# Now, evaluate the model on the test data
model.evaluate(test_gen)

32/32 [==============================] - 22s 669ms/step - loss: 0.6721 - accuracy: 0.8100


[0.6721240282058716, 0.8100000023841858]